In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")



In [2]:
def rfeFeature(indep_X,dep_Y,n):
    rfelist=[]
    log_model = LogisticRegression(solver='lbfgs')
    RF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
# NB = GaussianNB()
    DT= DecisionTreeClassifier(criterion = 'gini', max_features='sqrt',splitter='best',random_state = 0)
    svc_model = SVC(kernel = 'linear', random_state = 0)
#knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    rfemodellist=[log_model,svc_model,RF,DT] 
# Iterate through models and apply RFE
    for model in rfemodellist:
          print(f"Performing RFE with model: {model}")
        # Create the RFE model and select the top 'n' features
          log_rfe = RFE(model, n_features_to_select=n)
         # Fit the RFE model
          log_fit = log_rfe.fit(indep_X, dep_Y)
          log_rfe_feature=log_fit.transform(indep_X)
          rfelist.append(log_rfe_feature)
          #Get selected feature columns
          selected_features = indep_X.columns[log_rfe.support_]
#Print the selected features
          print("Selected features column: ", selected_features)

    return rfelist


In [3]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        #X_train, X_test, y_train, y_test = train_test_split(indep_X,dep_Y, test_size = 0.25, random_state = 0)
        
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        return X_train, X_test, y_train, y_test
    

In [4]:
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
     # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
    #from sklearn.metrics import confusion_matrix
    #cm = confusion_matrix(y_test, y_pred)
     Accuracy=accuracy_score(y_test, y_pred )
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

In [5]:
def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    

In [6]:
def svm_linear(X_train,y_train,X_test):
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [7]:
def svm_NL(X_train,y_train,X_test):
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [8]:
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    

In [9]:
def knn(X_train,y_train,X_test):
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [10]:
def Decision(X_train,y_train,X_test):
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm    

In [11]:
def random(X_train,y_train,X_test):
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    

In [12]:
def rfe_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf): 
    
    rfedataframe=pd.DataFrame(index=['Logistic','SVC','Random','DecisionTree'],columns=['Logistic','SVMl','SVMnl',
                                                                                        'KNN','Navie','Decision','Random'])
    for number, idex in enumerate(rfedataframe.index):
        # Use .loc to assign values based on row index and column name
        rfedataframe.loc[idex, 'Logistic'] = acclog[number]
        rfedataframe.loc[idex, 'SVMl'] = accsvml[number]
        rfedataframe.loc[idex, 'SVMnl'] = accsvmnl[number]
        rfedataframe.loc[idex, 'KNN'] = accknn[number]
        rfedataframe.loc[idex, 'Navie'] = accnav[number]
        rfedataframe.loc[idex, 'Decision'] = accdes[number]
        rfedataframe.loc[idex, 'Random'] = accrf[number]
    return rfedataframe

   



In [13]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True,dtype=int)
indep_X=df2.drop('classification_yes',axis=1)
dep_Y=df2['classification_yes']


In [14]:
df2

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [15]:
rfelist=rfeFeature(indep_X,dep_Y,3)       
acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]


    

Performing RFE with model: LogisticRegression()
Selected features column:  Index(['sg_c', 'sg_d', 'htn_yes'], dtype='object')
Performing RFE with model: SVC(kernel='linear', random_state=0)
Selected features column:  Index(['sg_d', 'dm_yes', 'appet_yes'], dtype='object')
Performing RFE with model: RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)
Selected features column:  Index(['sc', 'hrmo', 'rc'], dtype='object')
Performing RFE with model: DecisionTreeClassifier(max_features='sqrt', random_state=0)
Selected features column:  Index(['sc', 'pcv', 'sg_d'], dtype='object')


In [16]:
rfelist

[array([[1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 0.],
        ...,
        [1., 0., 1.],
        [0., 0., 1.],
        [0., 0., 0.]]),
 array([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        ...,
        [0., 1., 0.],
        [0., 1., 1.],
        [0., 0., 1.]]),
 array([[ 3.07735602, 12.51815562,  4.70559701],
        [ 0.7       , 10.7       ,  4.70559701],
        [ 0.6       , 12.        ,  4.70559701],
        ...,
        [ 6.        ,  9.1       ,  3.4       ],
        [ 6.8       ,  8.5       ,  4.70559701],
        [ 1.        , 16.3       ,  4.9       ]]),
 array([[ 3.07735602, 38.86890244,  0.        ],
        [ 0.7       , 34.        ,  0.        ],
        [ 0.6       , 34.        ,  0.        ],
        ...,
        [ 6.        , 26.        ,  0.        ],
        [ 6.8       , 38.86890244,  0.        ],
        [ 1.        , 53.        ,  0.        ]])]

In [17]:
for i in rfelist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)   
    
        
    classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
    acclog.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
    accsvml.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
    accsvmnl.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
    accknn.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
    accnav.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
    accdes.append(Accuracy)
    
    classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
    accrf.append(Accuracy)
    
result=rfe_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)



In [18]:
result
#3

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
Logistic,0.94,0.94,0.94,0.94,0.94,0.94,0.94
SVC,0.87,0.87,0.87,0.87,0.87,0.87,0.87
Random,0.91,0.92,0.93,0.93,0.86,0.91,0.94
DecisionTree,0.93,0.93,0.94,0.95,0.74,0.95,0.97
